ALL THE EXTERNAL REUIQREMENTS ARE PRESNET IN ./requirement.txt

In [ ]:
%pip install -r requirement.txt

In [ ]:
#error can occur, careful!!
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.0.0/en_core_web_md-3.0.0.tar.gz
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

Lets scrape the required dataset 
Input will be a text file containing articles abstracts from pubmed

In [1]:
from my_packages.get_document import module_get_document

kg_disease= str(input("please enter the disease you want a knowledge graph for: "))
no_of_articles= int(input("please enter the number of articles you want to consider: ")) #expected wall time for 10 articles around 12 mins


module_get_document.get_docx(kg_disease, no_of_articles)


Search results saved to a text file.


Lets crete a log file for ouputs 

In [3]:
with open("D:/VIISem/knowledgeGraphUsingMondo/ouput/output_log.txt", 'w') as fh:
    pass

Lets create a dataframe that can be later deployed to neo4J

In [4]:
from my_packages.get_knowledgeGraph import module_getKnowledgeGrpah

df= module_getKnowledgeGrpah.get_gph()


d:\VIISem\knowledgeGraphUsingMondo\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
100%|██████████| 110/110 [03:54<00:00,  2.13s/it]


Lets deploy df to neo4J :)

In [ ]:
from my_packages.create_graph import module_createGraph

class_labels= module_createGraph.load_gh("bolt://54.166.113.35:7687", "runaways-operator-ovens", df)

* Ignore below if you dont need to check efficiency!!

In [1]:
from my_packages.check_efficiencyGraph import module_get_efficiency
module_get_efficiency.check_ef("D:/VIISem/knowledgeGraphUsingMondo/ouput/dataframe_srt.csv", "D:/VIISem/knowledgeGraphUsingMondo/ouput/annotated.csv")

checking for original-> output
Percentage of source values matched: 51.35135135135135
Percentage of target values matched: 0.0
